# TSA Chapter 8: Case Study - ACF Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch8/TSA_ch8_case_acf_analysis/TSA_ch8_case_acf_analysis.ipynb)

ACF analysis of EUR/RON returns vs squared returns: short memory in returns, long memory in volatility.

In [ ]:
!pip install numpy pandas matplotlib statsmodels yfinance -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

In [ ]:
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
# Download EUR/RON exchange rate data
try:
    eurron_raw = yf.download('EURRON=X', start='2019-01-01', end='2025-01-01', progress=False)['Close'].squeeze()
    eurron_raw = eurron_raw.dropna()
    if len(eurron_raw) < 200:
        raise ValueError('Insufficient data')
    df = pd.DataFrame({'EURRON': eurron_raw.values.flatten()}, index=eurron_raw.index)
    print(f'EUR/RON data: {len(df)} observations')
except Exception as e:
    print(f'Download failed ({e}), using fallback')
    np.random.seed(42)
    n_obs = 1000
    dates = pd.date_range(start='2020-01-01', periods=n_obs, freq='B')
    base_rate = 4.85
    trend = np.linspace(0, 0.15, n_obs)
    seasonality = 0.02 * np.sin(2 * np.pi * np.arange(n_obs) / 252)
    noise = np.cumsum(np.random.randn(n_obs) * 0.002)
    eurron = base_rate + trend + seasonality + noise
    df = pd.DataFrame({'EURRON': eurron}, index=dates)

# Compute returns and squared returns
returns = df['EURRON'].pct_change().dropna() * 100
squared_returns = returns ** 2
acf_returns = acf(returns, nlags=40)
acf_squared = acf(squared_returns, nlags=40)

# Plot ACF comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 4.5))

axes[0].bar(range(len(acf_returns)), acf_returns, color=BLUE, width=0.6, alpha=0.7, edgecolor='white', linewidth=0.3)
conf = 1.96 / np.sqrt(len(returns))
axes[0].axhline(y=conf, color=RED, linestyle='--', linewidth=1)
axes[0].axhline(y=-conf, color=RED, linestyle='--', linewidth=1)
axes[0].set_xlabel('Lag')
axes[0].set_ylabel('ACF')
axes[0].set_title('ACF of Returns', color=BLUE, fontweight='bold')
axes[0].set_xlim(-1, 41)

axes[1].bar(range(len(acf_squared)), acf_squared, color=GREEN, width=0.6, alpha=0.7, edgecolor='white', linewidth=0.3)
axes[1].axhline(y=conf, color=RED, linestyle='--', linewidth=1)
axes[1].axhline(y=-conf, color=RED, linestyle='--', linewidth=1)
axes[1].set_xlabel('Lag')
axes[1].set_ylabel('ACF')
axes[1].set_title('ACF of Squared Returns (Volatility)', color=GREEN, fontweight='bold')
axes[1].set_xlim(-1, 41)

fig.legend(['95% Confidence Bounds'], loc='upper center', bbox_to_anchor=(0.5, -0.02),
           ncol=1, frameon=False)
plt.tight_layout()
save_chart(fig, 'ch8_case_acf_analysis')
plt.show()